# init AD9174 with external sampling clock
Direct drive of the DAC. No additonal VCO. Best configuration for phase noise.

Needs an external 5.12 GHz DAC sampling clock connected to J34 (`CLKIN`) of the `AD9174-FMC_EBZ` board. Make sure C34 and C35 are populated. Connect J3 and J41 with a short SMA cable. See `../doc/dac_clocking.png` for details.

requires litex_server running and connected to the VC707 USB-uart port:

```bash
litex_server --uart --uart-port /dev/ttyUSB0 --uart-baudrate 115200

```

In [1]:
import sys
from numpy import *
from time import sleep
from litex import RemoteClient
from ad9174 import Ad9174Settings, Ad9174Init
from hmc7044 import Hmc7044
from client_tools import getId, setSamples, big_write, big_read, hd

In [2]:
settings = Ad9174Settings(json_file='../build/csr.json')
print(settings)

----------------
 JESD mode 20
----------------
INTERP_CH: 1  INTERP_MAIN: 1  DSP_CLK_DIV: 16
JESD204BSettings(): 5a 05 00 87 00 1f 00 0f 2f 23 80 00 00 e6 
         DID:  90        BID:   5     ADJCNT:   0        LID:   0 
       PHADJ:   0     ADJDIR:   0          L:   8        SCR:   1 
           F:   1          K:  32          M:   1          N:  16 
          CS:   0         NP:  16  SUBCLASSV:   1          S:   4 
       JESDV:   1         CF:   0         HD:   1       RES1:   0 
        RES2:   0       FCHK: 230 
   [ LINK_DW:  32     FR_CLK:   4 ]


In [3]:
r = RemoteClient(csr_csv='../build/csr.csv', debug=False, port=1234)
r.open()
getId(r)

'AD9174 + VC707 test 2021-03-11 10:29:22'

In [4]:
ad = Ad9174Init(r, settings)
wr = ad.ad.wr
rr = ad.ad.rr

# 1. init AD9174

In [5]:
ad.init_ad9174(ADC_CLK_DIV=4, USE_PLL=False)

AD917X_NVM_BLR_DONE: 1
PROD_ID: 0x9174
PROD_GRADE: 0  DEV_REVISION: 5
DLL locked: 1
SPI_PAGEINDX: 0b01000001
CAL_STAT: 1
SERDES PLL locked: 1
MODE_NOT_IN_TABLE: 0


# 2. init HMC7044

In [6]:
hmc = ad.hmc
hmc.init_hmc7044_ext()

clk_div = ad.settings.DSP_CLK_DIV // 4
hmc.setup_channel(12, clk_div, sync_en=False)     # DEV_CLK to the FPGA

# for litejesd, SYSREF must be an integer multiple of the LMFC
lmfc_cycles = ad.settings.K // ad.settings.FR_CLK
hmc.setup_channel(3, clk_div * lmfc_cycles * 10)   # SYSREF (DAC)
hmc.setup_channel(13, clk_div * lmfc_cycles * 10)  # SYSREF (FPGA)
# hmc.trigger_reseed()
hmc.trigger_div_reset()

In [7]:
ad.fpga_print_clocks()

f_jesd = 312.501335 MHz  f_ref = 3.906267 MHz


# 3. Init the FPGA side

In [8]:
# r.regs.ctrl_reset.write(1)  # resets ALL clockdomains (HARSH!)
r.regs.control_control.write(0b01)  # resets PHYs and jesd core
print('status: {:03b}'.format(r.regs.control_status.read()))

# bit1: links_enable,  bit0: phys_reset
r.regs.control_control.write(0b10)
# bit2: /jsync,  bit1: links_ready,  bit0: phys_ready
print('status: {:03b}'.format(r.regs.control_status.read()))

ad.trigger_jref_sync()
ad.print_irq_flags(True, True)
if ad.print_irq_flags(True):
    print('😭')
else:
    print('😃')

status: 000
status: 111
SYNC_ROTATION_DONE 1
DYN_LINK_LATENCY  2 cycles
😃


# 4. Test link

In [9]:
ad.print_irq_flags(True)
print()

ad.print_ilas()

ad.print_lane_status()
print()

ad.test_stpl()


JESD settings, received on lane 0 vs (programmed):
450: 5a (5a)
451: 05 (05)
452: 00 (00)
453: 87 (87)
454: 00 (00)
455: 1f (1f)
456: 00 (00)
457: 0f (0f)
458: 2f (2f)
459: 23 (23)
45a: 80 (80)
45b: 00 (00)
45c: 00 (00)
45d: e6 (e6)
CHK: e6 (e6) 

Lane status:
      LANE_DESKEW: 11111111
    BAD_DISPARITY: 00000000
     NOT_IN_TABLE: 00000000
 UNEXPECTED_KCHAR: 00000000
    CODE_GRP_SYNC: 11111111
       FRAME_SYNC: 11111111
    GOOD_CHECKSUM: 11111111
   INIT_LANE_SYNC: 11111111
FIFO_STATUS_REG_0: 00000000
FIFO_STATUS_REG_1: 00000000
fpga j_sync errs: 4

STPL test:
converter: 0, sample: 0, tp: 597a, fail: 0
converter: 0, sample: 1, tp: b2f3, fail: 0
converter: 0, sample: 2, tp: 0c6c, fail: 0
converter: 0, sample: 3, tp: 65e5, fail: 0
converter: 0, sample: 4, tp: bf5e, fail: 0
converter: 0, sample: 5, tp: 18d7, fail: 0
converter: 0, sample: 6, tp: 7250, fail: 0
converter: 0, sample: 7, tp: cbc9, fail: 0
converter: 0, sample: 8, tp: 2542, fail: 0
converter: 0, sample: 9, tp: 7ebb, fail

0

# IQ or real sample waveforms
Output can be observed on IQ analyzer / scope.

In [5]:
# Positive and negative pulse
samples = zeros(4096, dtype=int16)
samples[100] = 32000
samples[140] = -32000

# 1 sine wave period over 100 samples
sin_data = array(sin(linspace(0, 2 * pi, 100)) * 15000, dtype=int16)
samples[200: 200 + len(sin_data)] = sin_data

# linear rise over 100 samples
lin_data = linspace(-10000, 10000, 100, dtype=int16)
samples[400: 400 + len(lin_data)] = lin_data

print(len(samples))
setSamples(r, samples)

4096


In [10]:
wr(0x596, (1 << 3) | (1 << 2))  # Turn ON Transmit enable

# Main DDS
internal frequency generator. Only works whn in a complex IQ channelizer mode.

In [35]:
# Setup DDSes
wr(0x1E6, (1 << 1))             # Enable DDSM_EN_CAL_DC_INPUT (see Fig. 80) (tone on / off)
wr(0x112, (1 << 3) | (1 << 2))  # Enable NCO + Modulus
wr(0x596, (1 << 3) | (1 << 2))  # Turn ON Transmit enable

# setup main DDS frequency and amplitude
ad.setTone(1, 1e9, 1, f_ref=5.12e9)

DC_CAL_TONE: ff 50 
DDSM_FTW: 00 00 00 00 00 32 
DDSM_FTW_LOAD_ACK: 1


In [36]:
# DC test-mode off, enable JESD input
ad.ad.wr(0x1e6, 0)

0

In [53]:
# Adjust DDS phase offset
ad.setTone(1, phase=0)

DDSM_NCO_PHASE_OFFSET: 00 00 


# app layer PRBS 😭
couldn't get it working 😠 this feature seems completely undocumented for AD9174 ☹️

Use PHY layer PRBS in `gtx_debug.ipynb` instead.

In [60]:
r.regs.prbs_gen_sample_prbs_en.write(0)

In [12]:
r.regs.prbs_gen_sample_prbs_en.read()

0

In [79]:
ad.ad.help(0x14b)

reg 0x14b, PRBS:
    bit 7, PRBS_GOOD_Q, R, reset 0x00
    DAC1 good data indicator. 1: Correct PRBS sequence detected. 0: Incorrect sequence detected. Sticky; reset to 1 by PRBS_RESET.

    bit 6, PRBS_GOOD_I, R, reset 0x00
    DAC0 good data indicator. 0: Incorrect sequence detected. Sticky; reset to 1 by PRBS_RESET. 1: Correct PRBS sequence detected.

    bit 5, RESERVED, R, reset 0x00
    Reserved.

    bit 4, PRBS_INV_Q, R/W, reset 0x01
    DAC1 data inversion. 0: Expect normal data. 1: Expect inverted data.

    bit 3, PRBS_INV_I, R/W, reset 0x00
    DAC0 data inversion. 0: Expect normal data. 1: Expect inverted data.

    bit 2, PRBS_MODE, R/W, reset 0x00
    Select which PRBS polynomial is used for the datapath PRBS test. 0: 7-bit: x7 + x6 + 1. 1: 15-bit: x15 + x14 + 1.

    bit 1, PRBS_RESET, R/W, reset 0x00
    Reset error counters. 0: Normal operation. 1: Reset counters.

    bit 0, PRBS_EN, R/W, reset 0x00
    Enable PRBS checker. 0: Disable. 1: Enable.



In [10]:
ad.ad.wr(0x14b, 0b0111)
ad.ad.wr(0x14b, 0b0101)

for ch in range(7):
    ad.ad.wr(0x14e, ch)
    print('{:d}: {:08b}, {:3d}, {:3d}'.format(ch, ad.ad.rr(0x14b), ad.ad.rr(0x14c), ad.ad.rr(0x14d)))

0: 00000101, 255, 255
1: 00000101, 255, 255
2: 00000101, 255, 255
3: 11000101,   0,   0
4: 11000101,   0,   0
5: 11000101,   0,   0
6: 00000101, 255, 255
